# Constructive heuristics comparison

The objective of the $\alpha$-neighbor $p$-center problem can be thought of as distributing the facilities among the clients to cover them efficiently, which is the actual goal of the $p$-center problem, so a constructive heuristic that uses its objective function will be tested and compared against a greedy heuristic that takes into account the objective function of this problem.

There will be used 20 random instances of size $n = 50$, $p = 5$ and other 20 of size $n = 400$, $p = 20$, and each one will be tested with both $\alpha = 2$ and $\alpha = 3$. The coordinates of the points are between 0 and 1000 for both planes.

In [39]:
from copy import deepcopy

from constructive_module import generate_instances

instances_n50_a2 = generate_instances(20, 50, 5, 2)
instances_n50_a3 = deepcopy(instances_n50_a2)
for i in instances_n50_a3:
    i.alpha = 3
instances_n50 = instances_n50_a2 + instances_n50_a3

instances_n400_a2 = generate_instances(20, 400, 20, 2)
instances_n400_a3 = deepcopy(instances_n400_a2)
for i in instances_n400_a3:
    i.alpha = 3
instances_n400 = instances_n400_a2 + instances_n400_a3

We will use the following code to measure the time taken by the evaluations and the objective function results, formatted in a Pandas DataFrame.

In [4]:
import timeit

import pandas as pd

from heuristics.constructive import pdp_based, greedy
from utils import eval_obj_func

def measure(instance, heuristic):
    start = timeit.default_timer()
    solution = heuristic(instance)
    time = timeit.default_timer() - start
    of = eval_obj_func(instance, solution)
    return heuristic.__name__, solution, of, time

def get_dataframe(data):
    return pd.DataFrame({
        colname: [d[i] for d in data]
        for colname, i in zip(
            ('n', 'p', 'a', 'heuristic', 'solution', 'OF', 'seconds'),
            range(len(data[0])))
    })

## Comparing data

In [64]:
pdp_data = [(*i.get_parameters(), *measure(i, pdp_based))  for i in instances_n50 + instances_n400]

pdp_df = get_dataframe(pdp_data)
pdp_df

,n,p,a,heuristic,solution,OF,seconds
0,50,5,2,pdp_based,"{17, 21, 22, 5, 13}",571,0.000591
1,50,5,2,pdp_based,"{18, 34, 25, 28, 29}",665,0.000481
2,50,5,2,pdp_based,"{32, 1, 39, 7, 25}",551,0.000503
3,50,5,2,pdp_based,"{18, 19, 14, 30, 31}",715,0.000502
4,50,5,2,pdp_based,"{48, 17, 36, 26, 31}",650,0.000495
...,...,...,...,...,...,...,...
75,400,20,3,pdp_based,"{196, 329, 9, 268, 18, 151, 152, 345, 219, 30,...",361,0.062079
76,400,20,3,pdp_based,"{323, 132, 70, 136, 331, 204, 141, 142, 11, 15...",364,0.070889
77,400,20,3,pdp_based,"{389, 390, 391, 264, 136, 330, 265, 72, 13, 20...",384,0.079459
78,400,20,3,pdp_based,"{130, 131, 9, 75, 338, 276, 277, 87, 27, 31, 9...",360,0.100802


Saving the evaluation to a CSV:

In [68]:
import os

OUT_FOLDER = 'nb_results\\constructive'
os.mkdir(OUT_FOLDER)
pdp_df.to_csv(os.path.join(OUT_FOLDER, 'pdp_df.csv'))

Calculating statistics of the results:

In [69]:
stats = {
    'pdp': {
        f'n{n}': {
            f'a{alpha}': pdp_df[
                (pdp_df['n'] == n) &
                (pdp_df['a'] == alpha)
                ][['OF', 'seconds']].describe()
            for alpha in (2, 3)
        }
        for n in (50, 400)
    }
}

In [76]:
stats

{'pdp': {'n50': {'a2':                OF    seconds
   count   20.000000  20.000000
   mean   656.450000   0.000591
   std     92.819278   0.000181
   min    551.000000   0.000465
   25%    597.250000   0.000495
   50%    641.000000   0.000501
   75%    688.750000   0.000624
   max    935.000000   0.001099,
   'a3':                OF    seconds
   count   20.000000  20.000000
   mean   874.300000   0.000887
   std     63.766102   0.000382
   min    744.000000   0.000468
   25%    841.250000   0.000497
   50%    875.500000   0.001056
   75%    923.750000   0.001134
   max    997.000000   0.001620},
  'n400': {'a2':                OF    seconds
   count   20.000000  20.000000
   mean   297.200000   0.087361
   std     14.954668   0.039577
   min    272.000000   0.056054
   25%    288.250000   0.065652
   50%    293.500000   0.075388
   75%    312.000000   0.099921
   max    328.000000   0.234872,
   'a3':                OF    seconds
   count   20.000000  20.000000
   mean   382.700000  